In [1]:
import pandas as pd
import spacy
import numpy as np

nlp = spacy.load("fr_core_news_sm")

# Objectif : trouver la question du thème la plus similaire

In [2]:
theme_quest_user = "Préparer mon séjour" # on fixe un theme au pif

In [3]:
#quest_user = "Puis-je évaluer le confort de mon domaine ?"
#quest_user = "Les annimaux domestiques sont-ils acceptés ?"
quest_user = "ai-je besoin d'un visa ou d'un passeport ?"

In [15]:
faq = pd.read_pickle('faq_centerPark.pkl')

### Nettoyage du corpus

In [16]:
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('french')

sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']
sw.sort()

def lemmatise_text(text):
    lst_lematised = [token.lemma_ for token in nlp(text)] 
    return ' '.join(lst_lematised).lower()


def stem_text(text):
    lst_stemmerised = [stemmer.stem(token) for token in word_tokenize(text)]    
    return ' '.join(lst_stemmerised)


def substitute_punctuation(text):
    return ' '.join(text.replace("'", ' ').translate(str.maketrans('', '', string.punctuation)).split())


def supp(text):
    return text.replace("«", "").replace("’", "").replace("•", "").replace("®", "")


def supp_sw(text):
    return ' '.join([token.text for token in nlp(text) if not token.text in sw])

In [17]:
faq['question_clean'] = (faq.question.apply(lemmatise_text)
                                                 .apply(stem_text)
                                                 .apply(substitute_punctuation)
                                                 .apply(supp)
                                                 .apply(supp_sw)
                              )
faq['reponse_clean'] = (faq.reponse.apply(lemmatise_text)
                                               .apply(stem_text)
                                               .apply(substitute_punctuation)
                                               .apply(supp)
                                               .apply(supp_sw)
                              )

In [18]:
faq.to_pickle('df_classif_similarity.pkl')

In [19]:
faq['tokens'] = faq['question_clean'].apply(nlp)

### sélection des questions du thème

In [259]:
faq_theme = faq[faq.theme == theme_quest_user][["question", 'reponse']]

### Nettoyage de la question

In [249]:
quest_user_clean = supp_sw(supp(substitute_punctuation(stem_text(lemmatise_text(quest_user)))))

### Similarité

In [250]:
quest_user_clean_tokens = nlp(quest_user_clean)

In [251]:
lst_similarity = [quest_user_clean_tokens.similarity(token) for token in faq_theme.tokens]

In [252]:
rep_quest_user = faq_theme.reponse[np.asarray(lst_similarity).argmax()]

In [253]:
rep_quest_user

'Il est possible d’arriver n’importe quel jour de la semaine, à l’exception du dimanche, et pour une durée de 2 à 7 nuits. Les possibilités d’arrivées pendant les périodes de vacances scolaires sont plus limitées. Vous pouvez vous référer aux possibilités d’arrivées et de séjour dans votre réservation.'